In [1]:
import pandas as pd
import numpy as np
from stopword import stopwords
stopw = stopwords.stopword()

ok


In [2]:
# data_ = pd.read_excel("hasil_prediksi\svm\seleksi/seleksi_all_fuzzy.xlsx") #hasil_prediksi\svm\seleksi seleksi_all_fuzzy_18_ribu
# data_ = pd.read_excel("hasil_prediksi\svm\seleksi/seleksi_all_fuzzy_18_ribu.xlsx")
data_ = pd.read_excel("data_latih/hasil_seleksi_New3.xlsx")
latih_cm = data_['komentar'].tolist()
label_cm = data_['label'].tolist()
data_.head()

,Unnamed: 0,label,komentar,komentar_kotor,kode salah
0,0,non spam,homwsti iri banget artis g ajak gengsi sian deh,@homwsti iri bnget artisnya g d ajak acara ber...,0
1,1,non spam,tulis email nama depan dr huruf s,Yang nulis email pasti nama depannya dr huruf S,0
2,2,non spam,salfok sama telinga,Salfok sama telinganya😹😹,0
3,3,non spam,egapratiwwi youtube guankirim lu nan,@egapratiwwi youtubenya udh guankirim ke lu hahha,0
4,4,non spam,abang radit mahh emang keren terimakasih bela ...,"Abang radit mahh emang keren,,,,👏👏👏👍👍👍,,,terim...",0


In [3]:
pd.value_counts(data_['label']).plot.bar(title='')
pd.value_counts(data_['label'])

non spam    56562
spam         9707
Name: label, dtype: int64

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump
#membuat TF-IDF
tfidf = TfidfVectorizer(stop_words = stopw+['nya','saya','kak'])
tfidf_matrix_save = tfidf.fit(latih_cm)
tfidf_matrix_ = tfidf.fit_transform(latih_cm) #For CM
fitur = tfidf.get_feature_names()
print(tfidf_matrix_.shape)
X_ = tfidf_matrix_#.toarray()
Y_ = np.array(label_cm)
#y = np.array(kode)

dump(tfidf_matrix_save, 'model/cnb/tfdf_model_60.joblib')
# dump(tfidf_matrix_save, 'model/gnb/tfdf_model_18.joblib')

(66269, 58616)


['model/cnb/tfdf_model_60.joblib']

In [ ]:
import json
with open("model/fitur.json", "w") as f:
    json.dump(fitur,f)
frekueni_spam = dict()
frekueni_nonspam = dict()
frekueni = dict()

for i in fitur:
    frekueni_spam.update({i:0})
    frekueni_nonspam.update({i:0})
    frekueni.update({i:{"spam":0, "non spam":0}})
    
for latih, label in zip(latih_cm, label_cm):
    if label == "spam":
        for i in latih.split():
            if i in fitur:
#                 print(i)
                frekueni_spam[i] +=1
                frekueni[i]["spam"] +=1
            
    elif label == "non spam":
        for i in latih.split():
            if i in fitur:
#                 print(i)
                frekueni_nonspam[i] +=1
                frekueni[i]["non spam"] +=1
with open("model/frequensi.json", "w") as f:
    json.dump(frekueni,f)
del frekueni_spam 
del frekueni_nonspam 
del frekueni 


In [9]:
alpha = 13.4

In [10]:
K = 10 #pembagian data uji dan data latih
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score #, precision_score, recall_score
kf= KFold(n_splits=K) 
skf = StratifiedKFold(n_splits=K,random_state=2, shuffle=True)
skf

StratifiedKFold(n_splits=10, random_state=2, shuffle=True)

In [11]:
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB

In [12]:
comNB = ComplementNB(alpha = alpha)
clf = GaussianNB(var_smoothing = 1)

# akurasi_svm = []
akurasi_cnb = []
akurasi_gnb = []
ii = 0
for train_index, test_index in skf.split(X_, Y_):
    print(ii+1, end="")
    ii+=1
    #membagi data latih dan data uji
    X_train, X_test = X_[train_index], X_[test_index] #pembagian data latih
    y_train, y_test = Y_[train_index], Y_[test_index]
    
    #Melatih Model
    mCNB = comNB.fit(X_train, y_train)
    mGNB = clf.fit(X_train, y_train)
    print("p ", end="-")
    #proses prediksi
#     P_SVM = mSVM.predict(X_test)
    P_CNB = mCNB.predict(X_test)
    P_GNB = mGNB.predict(X_test)
    
    #menyimpan hasil akurasi tiap iterasi
    akurasi_cnb.append(accuracy_score(y_test, P_CNB))
    akurasi_gnb.append(accuracy_score(y_test, P_GNB))
    
# rerata_akurasi_svm = sum(akurasi_svm)/len(akurasi_svm)
rerata_akurasi_cnb = sum(akurasi_cnb)/len(akurasi_cnb)
rerata_akurasi_gnb = sum(akurasi_gnb)/len(akurasi_gnb)

print("")
print("----------------------------------------")
print("Hasil K-Fold GNB")
print("   akurasi   :",rerata_akurasi_gnb)
print("----------------------------------------")
print("Hasil K-Fold CNB")
print("     Aplha   :",alpha)
print("   akurasi   :",rerata_akurasi_cnb)

1p -2p -3p -4p -5p -6p -7p -8p -9p -10p -
----------------------------------------
Hasil K-Fold GNB
   akurasi   : 0.6869566359588066
----------------------------------------
Hasil K-Fold CNB
     Aplha   : 13.4
   akurasi   : 0.9245612126182552


In [8]:
comNB = ComplementNB(alpha = alpha)
clf = GaussianNB(var_smoothing = 1)

#Melatih Model untuk data secara keseluruhan (X dan Y)
mGNB = clf.fit(X_, Y_)
comNB = comNB.fit(X_, Y_) #CNB

#save models
dump(comNB, 'model/cnb/CNB_MODEL_18.joblib')
dump(mGNB, 'model/gnb/GNB_MODEL_18.joblib')

['model/gnb/GNB_MODEL_18.joblib']

In [4]:
format(1e-10, '9f')

' 0.000000'

In [13]:
del X_
del Y_

# del mGNB
# del comNB

del P_CNB
del P_GNB

In [42]:
len((mGNB.sigma_[0]))

24575

In [52]:
mGNB.class_prior_

array([0.81175395, 0.18824605])

In [10]:
import json

In [100]:
# mGNB.theta_[0]

# Save GNB

In [10]:
non_spam_gnb_s = list()
spam_gnb_s = list()
non_spam_gnb_t = list()
spam_gnb_t = list()
spam = list()
non_spam = list()
dict_gnb = dict()
dict_gnb_ = dict()
for ft, nspam_t, spam_t, nspam_s, spam_s in zip(fitur, mGNB.theta_[0],mGNB.theta_[1], mGNB.sigma_[0],mGNB.sigma_[1]):
#     print(ft,format(nspam_t, '10f'),format(spam_t, '8f'))
    non_spam_gnb_s.append(nspam_s)
    spam_gnb_s.append(spam_s)
    non_spam_gnb_t.append(nspam_t)
    spam_gnb_t.append(spam_t)
    nsp = (nspam_t**2)/nspam_s
    sp = (spam_t**2)/spam_s
    non_spam.append(format(nsp, 'f'))
    spam.append(format(sp, 'f'))
    dict_gnb_.update({ft:{"non spam":nsp, 'spam':sp}})
    dict_gnb.update({ft:{"theta":{"non spam":nspam_t, "spam":spam_t}, "sigma":{"non spam":nspam_s, "spam":spam_s}}})
with open("model/gnb/bobot_gnb.json", "w") as f:
    json.dump(dict_gnb,f)
with open("model/gnb/bobot_gnb2.json", "w") as f:
    json.dump(dict_gnb_,f)

In [117]:
dict_ = {
    "fitur":fitur,
    "non spam_t":non_spam_gnb_t,
    "spam_t":spam_gnb_t,
    "non spam_s":non_spam_gnb_s,
    "spam_s":spam_gnb_s,  
}

df = pd.DataFrame.from_dict(dict_)
df.to_excel("model/gnb/bobot.xlsx")

dict_ = {
    "fitur":fitur,
    "non spam":non_spam,
    "spam":spam,  
}

df = pd.DataFrame.from_dict(dict_)
df["non spam"] = df["non spam"].astype(float)
df["spam"] = df["spam"].astype(float)
df.to_excel("model/gnb/bobot_gnb.xlsx")

0.0

In [85]:
len(comNB.feature_log_prob_[0])

24575

# Save CNB

In [12]:
non_spam_cnb = list()
spam_cnb = list()
# non_spam_gnb_t = list()
# spam_gnb_t = list()

dict_cnb = dict()
for ft, nspam_t, spam_t in zip(fitur, comNB.feature_log_prob_[0],comNB.feature_log_prob_[1]):
#     print(ft,format(nspam_t, '10f'),format(spam_t, '8f'))
    non_spam_cnb.append(nspam_t)
    spam_cnb.append(spam_t)
    dict_cnb.update({ft:{"non spam":nspam_t, "spam":spam_t}})
with open("model/cnb/bobot_cnb.json", "w") as f:
    json.dump(dict_cnb,f)

In [13]:
dict_ = {
    "fitur":fitur,
    "non spam":non_spam_cnb,
    "spam":spam_cnb,  
}

df = pd.DataFrame.from_dict(dict_)
# df["non spam"] = df["non spam"].astype(float)
# df["spam"] = df["spam"].astype(float)
df.to_excel("model/cnb/bobot_cnb.xlsx")
df

,fitur,non spam,spam
0,00,10.124647,10.102271
1,003,10.145823,10.181946
2,003g,10.136915,10.208296
3,006,10.107668,10.208296
4,007,10.145823,10.105736
5,00778,10.145823,10.183286
6,00alamat,10.136260,10.208296
7,00cek,10.104838,10.208296
8,00cukup,10.129599,10.208296
9,00daftar,10.138478,10.208296


In [113]:
# dict_cnb